## Importer les librairies

In [ ]:
using CSV, DataFrames, Distributions, Gadfly, MLBase, Random, Statistics
using GLM
import StatsBase
using LinearAlgebra
include("functions.jl")
using Plots            # Pour la heatmap du KNN

## Visualiser les données

Avant toute chose, il est important de voir ce à quoi ressemble nos données et quelles sont les liens entre celles-ci pour essayer de mieux les comprendre. Pour ce faire, nous avons décidé de tracer les nuages de points pour chaque paire de variable explicatives.

<img id="myimage" src="pair.png" style=" height:500px; width:500px;">


<div class="img-zoom-container", style="overflow:auto; height:550px; width:5669px;">
  <img id="myimage" src="pair.png" style=" height:5669px; width:5669px;">>
</div>

In [ ]:
data = CSV.read("train.csv")

Gadfly.set_default_plot_size(150cm, 150cm)

matrix = Array{Plot}(undef, 10, 10)

column = names(data)[2:end-1]
i = 1
for c1 in column
    j = 1
    for c2 in column
        if (i == j)
            matrix[i,j] = Gadfly.plot(data, x = c1, Geom.histogram(bincount = 30), color = :diagnosis)
        else
            matrix[i,j] = Gadfly.plot(data, x = c1, y = c2, color = :diagnosis)
        end
        j+=1
    end 
    i+=1
end 

#grid = gridstack(matrix) # graphing is so computationnally hard that it makes the jupyter bug
println("Graphing is done")

Nous remarquons plusieurs choses, premièrement, notre rayon semble avoir une relation directement proportionnelle avec le périmètre. De plus, l’air semble lié au rayon, par une relation exponentielle. Une forte relation exponentielle semble aussi exister entre la dimension fractale et le rayon. Nous voyons donc qu’il y aura forcément présence de colinéarité que nous pouvons éliminer à l’aide de la PCA.

# Section 1: KNN

Le K-nearest neighbors algorithm est un algorithme de classification assez simple. Il se base sur le fait que les features points d'une classe devraient, généralement, se trouver proche des autres points de la même classe (ils devraient être clustered si nous normalisons les données). En se basant sur ce principe, un KNN va tout simplement prendre ces prédictions sur la distance que le point du testing set a avec les points du training set. Une fois que ces distances se font calculer, il faudra prendre les K points qui sont les plus proches du point à prédire. Grâce aux K points trouvés et à leurs classes respectives, nous allons pouvons classifier nos nouveaux points (les points du testing set).

Avant de réaliser le KNN, il faut d'abord faire une analyse en composantes principales.

## Traiter les données (PCA)

In [ ]:
data = CSV.read("train.csv")
y_train = data[end]


X_train = data[2:end-1] #omit id and diagnosis

data_test = CSV.read("test.csv")
X_test = data_test[2:end]

id_test = data_test[1]


println(size(X_train))
println(size(X_test))

length_train = size(X_train)[1]
length_test = size(X_train)[1]


new_X = vcat(X_train, X_test)

### Normalisation
Nous avons testé un différent type de normalisation pour voir comment cela influait sur nos résultats. Par défaut, la fonction Normalize de Julia normalise en utilisant le Z-score. Mais il existe aussi le type UnitRangeTransform qui permet de normaliser nos données en donnant aux points extrêmes la même valeur pour chaque composante, malheureusement cela n’a pas amélioré notre score F1. 

Nous avons également recoder le Robust Scaler de la librairie Scikit Learn. Ce type de normalisation utilise plutôt l’écart interquartile, de sorte qu’il est robuste aux valeurs aberrantes. Voici l’équation de normalisation permettant de trouver la valeur normaliser $x_i$ à partir de $x'_i$.
$$x'= \frac{X_i - Q_1}{Q_3 - Q_1} $$

Cette tentative n’a, elle non-plus pas amélioré nos résultats. Nous aurions également pu essayer d’autres méthodes de normalisation, mais nous avons préféré passer notre temps à explorer d’autres modèles.

In [ ]:
X = convert(Array{Float64}, new_X)

Z = standardize(X)

# Décomposition en valeurs singulières de la matrice rectangulaire Z
F = svd(Z)

# Extraction de la matrice U
U = F.U

# Extraction de la matrice V
V = F.V

# Extraction des valeurs singulières
γ = F.S

## Faire le modèle KNN

Premièrement, le KNN que nous avons implémenté utilise la distance euclidienne entre les points (aussi connu sous le nom de "L2 norm"). C'est important de le mentionné, car il y a plus d'une façon de faire le calcul d'une distance. Toutefois, nous avons choisis celle-là vu qu'elle est plus populaire. Ensuite, ce qu'il faut savoir c'est qu'un KNN, en général, nécessite que ces entrées soient normalisé. En effet, cela est du au fait que nous trouvons la distance entre deux points et que si les entrées ne sont pas normalisés, alors une différence dans l'orde de grandeur des variables explicatives va jouer sur l'importance des variables (une variable avec une plus grande orde de grandeur pourrait sembler plus importante ce qu'elle est vraiment vu que les distances de cette variable seront plus grandes). Finalement, pour trouver un K qui devrait donner un bon score, il est courant de faire du K-fold cross validation avec le training data pour savoir quel K pourrait être bon avec le testing set (celui qui pourrait bien généraliser). De plus, nous avons limités notre recherche du K optimal à des K impairs pour la simple raison que nous voulons que la classification se fasse avec une majorité (ce qui pourrait ne pas arriver si la moitié des K points les plus proches soient d'une certaine classe et l'autre moitié de l'autre classe).

In [ ]:
function calculateDist(vec1, vec2)
    dist = 0
    taille = size(vec1)[1]
    for i = 1:taille
        dist += (vec1[i]-vec2[i])^2
    end
    
    return dist
end

function findMin(n, tab, used_indexes)
    mini_pos = 1
    mini = tab[mini_pos]
    
    for i = 2:length(tab)
        if (tab[i] < mini && !(i in used_indexes))
            mini_pos = i
            mini = tab[i]
        end
    end
    return mini_pos
end


function findIndexesNSmallest(n, tab)
    temp = copy(tab)
    indexes = []
    for i = 1:n
        new_index = findMin(n, temp, indexes)
        push!(indexes, new_index)
    end
    

    return indexes
end

function predict(k, X_train, y_train, X_test) #works best with odd k    
    nb_data = size(X_train)[1]
    
    distances = []
    for elem in 1:nb_data
        push!(distances, calculateDist(X_test, X_train[elem, :]))
    end
    
    indexes_distances = findIndexesNSmallest(k, distances)
    
    nb_0 = 0
    for index in indexes_distances
        if (y_train[index] == 0)
            nb_0 += 1
        end
    end
    
    return convert(Int8, (nb_0 < k - nb_0))
    
end

function knn(k, X_train, y_train, X_test)
    ans = []
    for elem in 1:size(X_test)[1]
        push!(ans, predict(k, X_train, y_train, X_test[elem, :]))
    end
    return ans
end

## K-fold cross validation

La validation croisée nous permettra d'utiliser toutes nos données comme ensemble de validation. Cela nous servira quand il sera temps d'évaluer la qualité du modèle.

In [ ]:
# K-cross validation
function findAllIndexes(length, nb_blocks)
    return [(convert(Int16, floor((i-1)*length/nb_blocks))+1, convert(Int16, floor(i*length/nb_blocks))) for i = 1:nb_blocks]
end

function countTFPN(t_label, predictions)
    TP, FP, FN, TN = 0, 0, 0, 0
    taille = size(t_label)[1]
    for i = 1:taille
        if(t_label[i] == 1 && predictions[i] == 1)
            TP += 1
        elseif(t_label[i] == 1 && predictions[i] == 0)
            FN += 1
        elseif(t_label[i] == 0 && predictions[i] == 1)
            FP += 1
        elseif(t_label[i] == 0 && predictions[i] == 0)
            TN += 1
        end
            
    end
    return TP, FP, FN, TN
end

function computeMetrics(t_label, predictions)
    TP, FP, FN, TN = countTFPN(t_label, predictions)
    precision, recall, accuracy = (TP/(TP + FP)), (TP/(TP + FN)), ((TP+TN)/(TP + FP + FN + TN))
    return [precision, recall, accuracy]
end

function split_train_test(X, y, index)
    sub_X_train = [X[1 : index[1]-1, :]; X[index[2] : end, :]]
    sub_X_test = X[index[1] : index[2], :]
    sub_y_train = [y[1 : index[1]-1]; y[index[2] : end]]
    true_y_test = y[index[1] : index[2]]
    return sub_X_train, sub_X_test, sub_y_train, true_y_test
end


function KCrossValidation(k, X, y, nb_blocks = 30)
    indexes = findAllIndexes(size(X)[1],nb_blocks)
    ans = []
    for interval = indexes
        sub_X_train, sub_X_test, sub_y_train, true_y_test = split_train_test(X, y, interval)
        sub_y_test =  knn(k, sub_X_train, sub_y_train, sub_X_test)
        push!(ans, computeMetrics(true_y_test, sub_y_test))
    end
    temp = mean(ans)
    return temp
end


## Trouver le K qui minimise l'erreur selon les métriques

Le K correspond au nombre de voisins considérés. On prendra celui qui donne le meilleur score F1.

In [ ]:
mat = γ[1] *U[:, 1]*V[:,1]'

X_train = mat[1:length_train, :]
X_test = mat[length_train+1:end, :]

println(size(X_train))
println(size(X_test))
println(size(y_train))

In [ ]:
spy_matrix = Array{AbstractFloat}(undef, 10, 10)
spy_matrix_text = Array{Plots.PlotText}(undef, 10, 10)
for j = 1:10
    println("Index : ", j)
    
    mat = γ[1] *U[:, 1]*V[:,1]'
    for k = 2:j
        mat += γ[k] *U[:, k]*V[:,k]'
    end
    

    X_train = mat[1:length_train, :]

    X_test = mat[length_train+1:end, :]
    metrics = []
    gen_temp = 1:2:19
    for i = 1:2:19
        temp = KCrossValidation(i, X_train, y_train)
        if (i ==  1)
            metrics = [[elem] for elem = temp]
        else
            taille = size(temp)[1]
            for index = 1:taille
                push!(metrics[index], temp[index])
            end
        end

    end
    ks = [i for i = gen_temp]

    
    precision = metrics[1]
    recall = metrics[2]
    for index = 1:length(precision)
        f1 = 2*(precision[index] * recall[index])/(precision[index] + recall[index])
        println("F1 score ",index*2 - 1,  "NN : ", f1)
        spy_matrix[index,j] = f1
        spy_matrix_text[index,j] = Plots.text(round(f1, digits=4),9)
    end
    
end


In [ ]:
Gadfly.set_default_plot_size(12cm, 12cm)
Axis_k = ["1","3","5","7","9","11","13","15","17","19"]
Axis_PCA = ["1","2","3","4","5","6","7","8","9", "10"]
p = heatmap(Axis_PCA,Axis_k,spy_matrix,
    ylabel ="Value of K in the KNN",
    xlabel = "Number of parameters conserved by the PCA",
    title = "F1 score in function of the hyperparameters",
    c=:Greens
)
annotate!( vec(tuple.((1:length(Axis_PCA))'.-0.5, (1:length(Axis_k)).-0.5, spy_matrix_text)) )

### Tentative et Amélioration possible 

Afin d’améliorer le résultat du KNN, nous avons expérimenté différentes stratégies qui avait le potentiel d'améliorer nos résultats en modifiant minimalement notre code. Par exemple, nous avons essayé d’autres métriques pour évaluer la distance. Pour, garder nos démarches simples, nous nous sommes contenté de tester les distances de Minkowski qui était très simples à coder à partir de notre fonction distance de base. 

Pour expliquer brièvement ces métriques, pour calculer la distance de Minkowski de degré P, il suffit de faire la somme des valeurs absolues des différences de composante pour chaque dimension de nos points et des mettre le résultat à la puissance P. 

Nous avons testé la distance de Minkowski de degré 1 à 4. De ces 4 tests, la distance de degré 2 qui correspond à la distance euclidienne classique donnait de loin le meilleur résultat. 

$$ D(Y,X) = (\sum{\lvert x_i - y_i \rvert}^p)^{\frac{1}{p}}$$

Nous avons également utilisé un KNN dont la valeur significative de chaque voisin était proportionnelle à l’inverse de la distance par rapport au point que nous voulons évaluer. De cette manière les voisins les plus proches ont une  plus grande influence sur la classification que les voisins éloignés. Malheureusement, encore une fois cette tentative a fait baisser notre score.

## Faire les prédictions sur le testing set 

Nous allons prendre un des K trouver précédemment qui avait obtenu un bon score.

In [ ]:
k = 5

mat = γ[1] *U[:, 1]*V[:,1]'
for i = 2:7
    mat += γ[i] *U[:, i]*V[:,i]'
end
X_train = mat[1:length_train, :]
X_test = mat[length_train+1:end, :]

y_test = knn(k, X_train, y_train, X_test)
prediction = DataFrame(id = id_test, diagnosis = y_test)
CSV.write("KNN.csv",prediction)

On en profite pour définir une fonction de prédiction plus générale, qui nous servira pour la section finale.

In [ ]:
function knn_predict(train, valid, k = 5, p = 7)
    y_train = train[end]
    X_train = train[2:end-1] #omit id and diagnosis
    X_test = valid[2:end]

    length_train = size(X_train)[1]

    new_X = vcat(X_train, X_test)
    
    X = convert(Array{Float64}, new_X)
    Z = standardize(X)

    # Décomposition en valeurs singulières de la matrice rectangulaire Z
    F = svd(Z)
    U = F.U
    V = F.V
    γ = F.S

    mat = γ[1] *U[:, 1]*V[:,1]'
    for i = 2:p
        mat += γ[i] *U[:, i]*V[:,i]'
    end
    X_train = mat[1:length_train, :]
    X_test = mat[length_train+1:end, :]

    return knn_counts(k, X_train, y_train, X_test)
end

### Conclusion de la section KNN

Le score obtenu avec le KNN est satisfaisant. Lorsque nous avons fait le K-fold cross validation nous avons pu voir que certaines valeurs de K et que certains nombres de paramètres conservés par le PCA obtiennent de meilleurs résultats que d'autres. Avec quelques essais sur Kaggle, nous avons déterminé que les hyper-paramètres qui donnent le meilleur résultat est lorsque le K = 5 et que le nombre de paramètres utilisés pour le PCA = 7. Le f1 score obtenu avec le K-fold cross validation est de 0.9344, mais lorsque nous l'avons rentré sur Kaggle, nous avons obtenu un score de 0.89743.

# Bayesien Naif

In [ ]:
Gadfly.set_default_plot_size(15cm, 10cm)

## 1. Chargement des données

Assurez vous d'avoir télécharger les données dans le répertoire de ce calepin.

In [ ]:
data = CSV.read("train.csv")
first(data,5)

In [ ]:
# Séparation du jeu de données
malign = filter(row -> row.diagnosis == 1, data)
benign = filter(row -> row.diagnosis == 0, data)
n₁ = size(malign, 1)
n₀ = size(benign, 1)
n = n₁ + n₀

## 2. Loi prédictive

Après avoir réalisé les calculs pour trouver la loi prédictive pour une loi normale considérant nos a priori non informatifs, on s'assurera qu'elle correspond aussi à une réalisation d'échantillonnage.

In [ ]:
loi_inconnue = Normal(5, 2)

In [ ]:
n = 150
y = rand.(loi_inconnue for i=1:n)

In [ ]:
ȳ = mean(y)
SST = sum((y .- ȳ).^2)
s = SST / (n-1)
mμ = LocationScale(ȳ, s/sqrt(n), TDist(n-1))
mσ² = InverseGamma((n-1)/2, 1/2*SST)
Gadfly.plot(
    layer(x -> pdf(mμ, x), 0, 10, color=["mu"]),
    layer(x -> pdf(mσ², x), 0, 10, color=["sigma"])
)

In [ ]:
fucked_up_variance = (n+1) * (SST) / (n * (n-2))
dist_pred = LocationScale(ȳ, sqrt(fucked_up_variance), TDist(n-2))

In [ ]:
# Gibbs avec densités marginales
quantity = 1000000
ỹ = zeros(quantity)
for i=1:quantity
    μ_rand = rand(mμ)
    σ_rand = sqrt(rand(mσ²))
    ỹ[i] = rand(Normal(μ_rand, σ_rand))
end

In [ ]:
Gadfly.plot(
    layer(x=y, alpha=[0.5],Geom.histogram(bincount=floor(sqrt(n)), density=true), Theme(default_color=colorant"brown")),
    layer(x -> pdf(dist_pred, x), -30, 30, Theme(default_color=colorant"black")),
    layer(x -> pdf(loi_inconnue, x), -30, 30, Theme(default_color=colorant"red")),
    layer(x=ỹ, Geom.histogram(bincount=floor(sqrt(quantity)), density=true)),
    Coord.cartesian(xmin=-5, xmax=15)
)

En noir : loi prédictive <br>
En rouge: loi inconnue <br>
En bleu: valeurs d'échantillonnage <br>

## 3. Application des lois aux données

Pour chaque attribut, on représentera la distribution prédictive.

In [ ]:
# Calcul du modèle
function loi_predictive(dataset::Array{Float64})
    ȳ = mean(dataset)
    n = length(dataset)
    SST = sum((dataset .- ȳ).^2)
    fucked_up_variance = (n+1) * (SST) / (n * (n-2))
    return LocationScale(ȳ, sqrt(fucked_up_variance), TDist(n-2))
end

In [ ]:
n = length(data[!, :id])

In [ ]:
tag = :radius
pred_radius_0 = loi_predictive(benign[!, tag])
pred_radius_1 = loi_predictive(malign[!, tag])
Gadfly.plot(
    layer(x->pdf(pred_radius_0, x), 0, 30, Theme(default_color=colorant"black")),
    layer(x->pdf(pred_radius_1, x), 0, 30, Theme(default_color=colorant"black")),
    layer(malign, x=tag, Geom.histogram(bincount=floor(sqrt(n₁)), density=true), color = ["malign"]),
    layer(benign, x=tag, alpha=[0.75], Geom.histogram(bincount=floor(sqrt(n₀)), density=true), color = ["benign"])
)

In [ ]:
tags = [:radius,:texture,:perimeter,:area,:smoothness,:compactness,
    :concavity,:concave_points,:symmetry,:fractal_dimension]
plots = []
preds_0 = []
preds_1 = []
for tag in tags
    pred_0 = loi_predictive(benign[!, tag])
    pred_1 = loi_predictive(malign[!, tag])
    push!(preds_0, pred_0)
    push!(preds_1, pred_1)
    max₀ = maximum(benign[!, tag])
    max₁ = maximum(malign[!, tag])
    push!(plots, Gadfly.plot(
        layer(x->pdf(pred_0, x), 0, 1.2 * max₀, Theme(default_color=colorant"black")),
        layer(x->pdf(pred_1, x), 0, 1.2 * max₁, Theme(default_color=colorant"black")),
        layer(malign, x=tag, Geom.histogram(bincount=floor(sqrt(n₁)), density=true), color = ["malign"]),
        layer(benign, x=tag, Geom.histogram(bincount=floor(sqrt(n₀)), density=true), color = ["benign"])
    ))
end

In [ ]:
plots[1]

In [ ]:
plots[2]

In [ ]:
plots[3]

In [ ]:
plots[4]

In [ ]:
plots[5]

In [ ]:
plots[6]

In [ ]:
plots[7]

In [ ]:
plots[8]

In [ ]:
plots[9]

In [ ]:
plots[10]

Si l'on utilise uniquement ces lois avec le modèle bayésien naïf (voir section 4), on obtient un score de 89% sur les données d'entrainement, et de 78% sur les données de test. On peux donc faire mieux!

## 3.2 Recherche de nouvelles lois

La loi normale semble convenir pour la plupart des attributs. Néanmoins, pour la compacité et la concavité, il est clair que de meilleures lois peuvent être définies.

Le temps nous étant compté, on utilisera cette fois la fonction *fit* pour approximer la loi a posteriori. Ainsi, au lieu d'utiliser la vraie loi prédicitve, on se servira d'estimateurs. On fera donc la même approximation que dans le devoir 8:

$$f_{(\tilde{X}_1|\tilde{Y}=0)}(\tilde{x}_1) \approx f_{(\tilde{X}_1|\tilde{Y}=0, \hat{\boldsymbol{\theta}}_{01})}(\tilde{x}_1)$$

On commencera avec les lois qui nous semblent les plus probables sur la compacité des tumeurs bénignes.

In [ ]:
gamma = fit(Gamma{Float64}, benign[!, :compactness])
beta = fit(Beta{Float64}, benign[!, :compactness])
lognormal = fit(LogNormal{Float64}, benign[!, :compactness])

In [ ]:
Gadfly.plot(
    layer(x -> pdf(gamma, x), 0, 0.5, color=["gamma"]),
    layer(x -> pdf(beta, x), 0, 0.5, color=["beta"]),
    layer(x -> pdf(lognormal, x), 0, 0.5, color=["lognormal"]),
    layer(x -> pdf(preds_0[6], x), 0, 0.5, Theme(default_color=colorant"black")),
    layer(x=benign[!, :compactness], Geom.histogram(bincount = floor(sqrt(n₀)), density=true), 
        Theme(default_color=colorant"lightblue")),
    Guide.xlabel("benign.compactness"), Guide.ylabel("density")
)

Ces trois lois semblent mieux décrire la compacité des tumeurs que la loi normale. Pour comparer les différents modèles, nous utiliserons le BIC.

In [ ]:
# Tous nos modèles ont deux paramètres
# k=2
df_bic = DataFrame(Model = String[], BIC=Float64[])
push!(df_bic, ["student", loglikelihood(preds_0[6], benign[!, :compactness]) - log(n₀)])
push!(df_bic, ["gamma", loglikelihood(gamma, benign[!, :compactness]) - log(n₀)])
push!(df_bic, ["beta", loglikelihood(beta, benign[!, :compactness]) - log(n₀)])
push!(df_bic, ["lognormal", loglikelihood(lognormal, benign[!, :compactness]) - log(n₀)])

Devançant la loi gamma de presque trois points, le modèle le plus probable pour nos données est la lognormale, ce qui semble correspondre à ce qu'on peut voir sur le graphique plus haut. Nous allons maintenant appliquer ces mêmes modèles aux autres attributs. Nous en profiterons aussi pour voir la qualité de la loi prédictive par rapport à la loi normale avec paramètres estimés.

In [ ]:
bic_malign = DataFrame(Student = Float64[], Normal=Float64[], Gamma=Float64[], Beta=Float64[], LogNormal=Float64[])
for i=1:10
    tag = tags[i]
    y = malign[:, tag]
    gammaValid = minimum(y) > 0
    betaValid = minimum(y) >= 0 && maximum(y) <=1

    push!(bic_malign, [
        loglikelihood(preds_1[i], y) - log(n₀),
        loglikelihood(fit(Normal, y), y) - log(n₀),
        gammaValid ? loglikelihood(fit(Gamma{Float64}, y), y) - log(n₀) : -Inf,
        betaValid ? loglikelihood(fit(Beta{Float64}, y), y) - log(n₀) : -Inf,
        gammaValid ? loglikelihood(fit(LogNormal{Float64}, y), y) - log(n₀) : -Inf
    ])
end
bic_malign

In [ ]:
bic_benign = DataFrame(Student = Float64[], Normal=Float64[], Gamma=Float64[], Beta=Float64[], LogNormal=Float64[])
for i=1:10
    tag = tags[i]
    y = benign[:, tag]
    gammaValid = minimum(y) > 0
    betaValid = minimum(y) >= 0 && maximum(y) <=1

    push!(bic_benign, [
        loglikelihood(preds_0[i], y) - log(n₀),
        loglikelihood(fit(Normal, y), y) - log(n₀),
        gammaValid ? loglikelihood(fit(Gamma{Float64}, y), y) - log(n₀) : -Inf,
        betaValid ? loglikelihood(fit(Beta{Float64}, y), y) - log(n₀) : -Inf,
        gammaValid ? loglikelihood(fit(LogNormal{Float64}, y), y) - log(n₀) : -Inf
    ])
end
bic_benign

On voit que pour certaines variables, notamment la compacité (index 6), le BIC penche décisivement en faveur des autres modèles (différence supérieure à 2). On remarque au passage que, pour un modèle utilisant la loi normale, le facteur de Bayes entre la loi prédictive et l'estimation par maximum de vraisemblance penche toujours en faveur de la loi prédictive, mais rarement de façon décisive.

Néanmoins, certaines des valeurs de concavité (index 7 et 8) sont en dessous de 0 dans les tumeurs bénignes. C'est plutôt dommage, surtout que visuellement, la loi normale était plutôt mauvaise pour modéliser ces valeurs! De plus, selon le BIC, ces modèles conviennent mieux aux valeurs des tumeurs malignes, comme on peut le voir ci-dessous:

In [ ]:
y = malign[:, :concavity]

gamma = fit(Gamma{Float64}, y)
beta = fit(Beta{Float64}, y)
lognormal = fit(LogNormal{Float64}, y)

Gadfly.plot(
    layer(x -> pdf(gamma, x), 0, 0.5, color=["gamma"]),
    layer(x -> pdf(beta, x), 0, 0.5, color=["beta"]),
    layer(x -> pdf(lognormal, x), 0, 0.5, color=["lognormal"]),
    layer(x -> pdf(preds_1[7], x - 0.01), 0, 0.5, Theme(default_color=colorant"black")),
    layer(x=y, Geom.histogram(bincount = floor(sqrt(n₀)), density=true), 
        Theme(default_color=colorant"lightblue")),
    Guide.xlabel("malign.concavity"), Guide.ylabel("density")
)

On tentera donc de gérer ces valeurs négatives, pour utiliser les lois proposées sur les données de concavité.

In [ ]:
minimum(benign[!, tags[7]])

In [ ]:
minimum(benign[!, tags[8]])

In [ ]:
count(val-> val <= 0, benign[:, :concavity])

In [ ]:
count(val-> val <= 0, benign[:, :concave_points])

Très peu de valeurs sont inférieures ou égales à 0, et celles qui le sont sont très proches de 0. On peut donc se permettre de les ramener à une valeur juste au-dessus de 0.

Il faudra toutefois se rappeler d'effectuer la même transformation aux données de test!

On commencera avec la concavité.

In [ ]:
y = benign[:, :concavity]
y = map(val -> val <= 0 ? 0.0001 : val, y)

gamma = fit(Gamma{Float64}, y)
beta = fit(Beta{Float64}, y)
lognormal = fit(LogNormal{Float64}, y)

Gadfly.plot(
    layer(x -> pdf(gamma, x), 0, 0.5, color=["gamma"]),
    layer(x -> pdf(beta, x), 0, 0.5, color=["beta"]),
    layer(x -> pdf(lognormal, x), 0, 0.5, color=["lognormal"]),
    layer(x -> pdf(preds_0[7], x), 0, 0.5, Theme(default_color=colorant"black")),
    layer(x=y, Geom.histogram(bincount = floor(sqrt(n₀)), density=true), 
        Theme(default_color=colorant"lightblue")),
    Guide.xlabel("benign.concavity"), Guide.ylabel("density")
)

In [ ]:
# Tous nos modèles ont deux paramètres
# k=2
df_bic = DataFrame(Model = String[], BIC=Float64[])
push!(df_bic, ["student", loglikelihood(preds_0[7], y) - log(n₀)])
push!(df_bic, ["gamma", loglikelihood(gamma, y) - log(n₀)])
push!(df_bic, ["beta", loglikelihood(beta, y) - log(n₀)])
push!(df_bic, ["lognormal", loglikelihood(lognormal, y) - log(n₀)])

Tous ces modèles sont nettements suprérieures à la loi normale. On remarque toutefois sur le graphique qu'ils ne semblent pas très bien suivre les données. On tentera donc un autre technique pour inclure les valeurs négative : décaler les données.

Le décalage de 0.01 a été choisi visuellement comme celui qui permet aux lois de mieux suivre les données. On pourrait faire un travail pour maximiser la vraisemblance de ce paramètre, mais pour l'instant, comme nous voulons vraiment juste améliorer nos prédictions par rapport à la loi normale, nous nous sommes contentés de cette approximation.

In [ ]:
decalage = 0.01
y = benign[:, :concavity] .+ decalage

gamma = fit(Gamma{Float64}, y)
beta = fit(Beta{Float64}, y)
lognormal = fit(LogNormal{Float64}, y)

Gadfly.plot(
    layer(x -> pdf(gamma, x), 0, 0.5, color=["gamma"]),
    layer(x -> pdf(beta, x), 0, 0.5, color=["beta"]),
    layer(x -> pdf(lognormal, x), 0, 0.5, color=["lognormal"]),
    layer(x -> pdf(preds_0[7], x - decalage), 0, 0.5, Theme(default_color=colorant"black")),
    layer(x=y, Geom.histogram(bincount = floor(sqrt(n₀)), density=true), 
        Theme(default_color=colorant"lightblue")),
    Guide.xlabel("benign.concavity"), Guide.ylabel("density")
)

In [ ]:
decalage = 0.01
y = benign[:, :concavity] .+ decalage
# Pour calculer le BIC avec ces nouvelles données, on devra adapter la loi prédictive
pred = loi_predictive(y)

df_bic = DataFrame(Model = String[], BIC=Float64[])
push!(df_bic, ["student", loglikelihood(pred, y) - log(n₀)])
# Le rajout du paramètre de décalage rajoute un paramètre à notre modèle!
# Cela n'affecte pas la loi normale, puisque ce paramètre correspond à la moyenne.
# k=3
push!(df_bic, ["gamma", loglikelihood(gamma, y) - 3/2 * log(n₀)])
push!(df_bic, ["beta", loglikelihood(beta, y) - 3/2 * log(n₀)])
push!(df_bic, ["lognormal", loglikelihood(lognormal, y) - 3/2 * log(n₀)])

La gamma de la première mise à niveau a à peu près le même BIC que la lognormae de la seconde. On peut donc choisir l'un ou l'autre de ces modèles.

On poursuit avec les points concaves. On appliquera la même démarche que pour la concavité.

In [ ]:
y = benign[:, :concave_points]
y = map(val -> val <= 0 ? 0.0001 : val, y)

gamma = fit(Gamma{Float64}, y)
beta = fit(Beta{Float64}, y)
lognormal = fit(LogNormal{Float64}, y)

Gadfly.plot(
    layer(x -> pdf(gamma, x), 0, 0.2, color=["gamma"]),
    layer(x -> pdf(beta, x), 0, 0.2, color=["beta"]),
    layer(x -> pdf(lognormal, x), 0, 0.2, color=["lognormal"]),
    layer(x -> pdf(preds_0[8], x), 0, 0.2, Theme(default_color=colorant"black")),
    layer(x=y, Geom.histogram(bincount = floor(sqrt(n₀)), density=true), 
        Theme(default_color=colorant"lightblue")),
    Guide.xlabel("benign.concave_points"), Guide.ylabel("density")
)

In [ ]:
# Tous nos modèles ont deux paramètres
# k=2
df_bic = DataFrame(Model = String[], BIC=Float64[])
push!(df_bic, ["student", loglikelihood(preds_0[8], y) - log(n₀)])
push!(df_bic, ["gamma", loglikelihood(gamma, y) - log(n₀)])
push!(df_bic, ["beta", loglikelihood(beta, y) - log(n₀)])
push!(df_bic, ["lognormal", loglikelihood(lognormal, y) - log(n₀)])

In [ ]:
y = benign[:, :concave_points]
y = y .+ 0.01

gamma = fit(Gamma{Float64}, y)
beta = fit(Beta{Float64}, y)
lognormal = fit(LogNormal{Float64}, y)

Gadfly.plot(
    layer(x -> pdf(gamma, x), 0, 0.2, color=["gamma"]),
    layer(x -> pdf(beta, x), 0, 0.2, color=["beta"]),
    layer(x -> pdf(lognormal, x), 0, 0.2, color=["lognormal"]),
    layer(x -> pdf(preds_0[8], x-0.01), 0, 0.2, Theme(default_color=colorant"black")),
    layer(x=y, Geom.histogram(bincount = floor(sqrt(n₀)), density=true), 
        Theme(default_color=colorant"lightblue")),
    Guide.xlabel("benign.concave_points"), Guide.ylabel("density")
)

In [ ]:
df_bic = DataFrame(Model = String[], BIC=Float64[])
push!(df_bic, ["student", loglikelihood(loi_predictive(y), y) - log(n₀)])
push!(df_bic, ["gamma", loglikelihood(gamma, y) - 3/2 * log(n₀)])
push!(df_bic, ["beta", loglikelihood(beta, y) - 3/2 * log(n₀)])
push!(df_bic, ["lognormal", loglikelihood(lognormal, y) - 3/2 * log(n₀)])

## 3.3 Choix du modèle

Donnant suite à notre analyse exploratoire, nous avons choisi les modèles que seront utlisés pour le bayésien naïf. Nous en avons aussi profité pour sélectionner les variables, en fonction de leur pertinence et de la colinéarité.

Les variables perimeter, area et concave_points seront retirées ainf de diminuer la multicolinéarité. On enlève aussi fractal_dimensions, qui n'est visuellement pas pertinente.

Cela nous laisse avec les variables suivantes, que nous modéliserons avec le modèle qui présente de le meilleur BIC. En cas de doute ($\Delta$BIC < 2), on choisit un modèle qui convient aux deux ensembles de données, par souci de cohérence.
- radius : Normale
- texture : LogNormale
- smoothness : LogNormale
- compactness : LogNormale
- concavity : Gamma (avec modification des valeurs négatives)
- symmetry : LogNormale

In [ ]:
variables = [1, 2, 5, 6, 7, 9]

for i=[2, 5, 6, 9]
    preds_0[i] = fit(LogNormal, benign[:, tags[i]])
    preds_1[i] = fit(LogNormal, malign[:, tags[i]])
end

preds_0[7] = fit(Gamma, map(val-> val <= 0 ? 0.0001 : val, benign[:, :concavity]))
preds_1[7] = fit(Gamma, malign[:, :concavity])

In [ ]:
for i=variables
    y₁ = malign[:, tags[i]]
    y₀ = benign[:, tags[i]]
    plots[i] = Gadfly.plot(
        layer(x->pdf(preds_0[i], x), 0, 1.2 * maximum(y₀), Theme(default_color=colorant"black")),
        layer(x->pdf(preds_1[i], x), 0, 1.2 * maximum(y₁), Theme(default_color=colorant"black")),
        layer(x=y₁, Geom.histogram(bincount=floor(sqrt(n₁)), density=true), color = ["malign"]),
        layer(x=y₀, Geom.histogram(bincount=floor(sqrt(n₀)), density=true), color = ["benign"])
    )
end

In [ ]:
plots[1]

In [ ]:
plots[2]

In [ ]:
plots[5]

In [ ]:
plots[6]

In [ ]:
plots[7]

In [ ]:
plots[9]

## 4. Probabilité d'une tumeur maligne

On utilisera un modèle bayésien naïf à plusieurs variables pour déterminer si une tumeur est maligne ou bénigne

In [ ]:
α = 1
β = 1
n₀ = size(benign, 1)
n₁ = size(malign, 1)
n = n₀ + n₁
p₀ = (β + n₀)/(α + β + n)
p₁ = (α + n₁)/(α + β + n)

In [ ]:
function vraisemblance_maligne(row::DataFrameRow, selected_variables)
    q₀ = log(p₀)
    q₁ = log(p₁)
    
    for i=selected_variables
        q₀ += log(pdf(preds_0[i], row[tags[i]]))
        
        q₁ += log(pdf(preds_1[i], row[tags[i]]))
    end

    return q₁ - q₀
end

In [ ]:
results = map(row -> vraisemblance_maligne(row, variables), eachrow(data))

In [ ]:
predictions = results .> 0

In [ ]:
mean(predictions)

In [ ]:
correctrate(data[!, :diagnosis], predictions)

### 4.1 Cross validation

Nous nous sommes demandés si enlever des variables colinéaires améliorait effectivement les prédictions. Pour cela, il faut séprarer notre ensemble de données entre entrainement et validation, puisque si l'utilisation de toutes les variables va toujours améliorer notre précision sur l'ensemble d'entrainement, elle peut la réduire sur une ensemble de test.

Il faut aussi choisir des lois pour les autres variables que nous avions rejetées.

- perimeter : Normale
- area : Normale
- concave_points : Gamma avec recentrage des données
- fractal_dimension : LogNormale

In [ ]:
function bayesian_predict(m_train, b_train, valid, variables)
    # Lois normales
    for attr=[1, 3, 4]
        preds_0[attr] = loi_predictive(b_train[:, tags[attr]])
        preds_1[attr] = loi_predictive(m_train[:, tags[attr]])
    end

    # Lois lognormales
    for attr=[2, 5, 6, 9, 10]
        preds_0[attr] = fit(LogNormal, b_train[:, tags[attr]])
        preds_1[attr] = fit(LogNormal, m_train[:, tags[attr]])
    end

    # Lois gamma
    # Pour la concavité des tumeurs bénignes, on modifie les points sous 0
    # On n'applique pas ces modifications aux tumeurs malignes, puisqu'aucune valeur ne se trouve sous 0
    y = map(val -> val <=0 ? 0.00001 : val, b_train[:, :concavity])
    preds_0[7] = fit(Gamma, y)
    preds_1[7] = fit(Gamma, m_train[:, :concavity])
    # Pour les points concaves des tumeurs bénignes, on décale tout de 0.01
    # Ces modifications devront aussi s'appliquer aux tumeurs malignes
    preds_0[8] = fit(Gamma, b_train[:, :concave_points] .+ 0.01)
    preds_1[8] = fit(Gamma, m_train[:, :concave_points] .+ 0.01)

    # Il faut appliquer les mêmes changements à l'ensemble de test
    valid = copy(valid)
    valid[!, :concavity] = map(val -> val <=0 ? 0.00001 : val, valid[:, :concavity])
    valid[!, :concave_points] = valid[:, :concave_points] .+ 0.01

    results = DataFrame(id = Int64[], predictions = Float64[])
    map(row->push!(results, [row.id, vraisemblance_maligne(row, variables)]), eachrow(valid))
    return results
end

On réutilisera la méthode `findAllIndexes` du KNN.

In [ ]:
function bayesian_split_data(i::Int64, nb_blocks::Int64)
    malign_indexes = findAllIndexes(n₁, nb_blocks)
    benign_indexes = findAllIndexes(n₀, nb_blocks)
    
    m_range = malign_indexes[i][1]:malign_indexes[i][2]
    malign_valid = malign[m_range, :]
    malign_train = malign[Not(m_range), :]

    b_range = benign_indexes[i][1]: benign_indexes[i][2]
    benign_valid = benign[b_range, :]
    benign_train = benign[Not(b_range), :]
    
    valid = append!(malign_valid, benign_valid)
    
    return malign_train, benign_train, valid
end

On peut maintenant appliquer nos modèles à chaque bloc

In [ ]:
rates_all = []
rates_selected = []
selected_variables = [1, 2, 5, 6, 7, 9]
nb_blocks = 5
for i=1:nb_blocks
    m_train, b_train, valid = bayesian_split_data(i, nb_blocks)
    
    # Prédictions avec variables sélectionnées
    results = bayesian_predict(m_train, b_train, valid, selected_variables)
    predictions = results.predictions .> 0
    push!(rates_selected, correctrate(predictions, valid.diagnosis))
    
    # Prédictions avec toutes les variables
    results = bayesian_predict(m_train, b_train, valid, 1:10)
    predictions = results.predictions .> 0
    push!(rates_all, correctrate(predictions, valid.diagnosis))
end

In [ ]:
mean(rates_all)

In [ ]:
mean(rates_selected)

Notre modèle ne semble pas être trop sensible à la colinéarité, ou aux variables presque inutiles. On peut donc se peremttre d'utiliser l'ensemble des variables pour le modèle final.

### 4.2 Application du modèle à l'ensemble de test

In [ ]:
test = CSV.read("test.csv")
first(test, 5)

In [ ]:
# Cette fois-ci, l'ensemble d'entrainement repose sur toutes les données disponibles
results = bayesian_predict(malign, benign, test, 1:10)
first(results, 5)

In [ ]:
predictions = results.predictions .> 0
mean(predictions)

In [ ]:
final = DataFrame(id = test[:,:id], diagnosis = predictions)
CSV.write("bayes_naif.csv",final)

Ce modèle a obtenu une précision de 80% sur l'ensemble de test ¯\\\_(ツ)_/¯

## 5. Analyse critique

Le travail sur la loi prédictive n'était pas vraiment nécessaire finalement, puisque d'autres modèles que la loi normale se sont assez souvent révélés de meilleure qualité, et même quand la loi normale est le meilleure modèle, la loi prédictive n'est pas significativement meilleure que l'estimation des paramètres d'après le BIC.

À bien y penser, il n'est peut-être  pas très bon de séparer l'ensemble de données entre tumeurs malignes et bénignes avant même de faire la validation croisée. Cette séparation a été notamment faite pour ne pas avoir besoin de recalculer p₀ et p₁, même si leur valeur n'est plus exacte pour la validation, et pour éviter d'avoir un cas extrême où il y a très peu de valeurs dans l'une ou l'autre des catégories.

# Section 4: Régression logistique

Nous avons déterminés lors de l'analyse de composantes/l'analyse de multicolinéarité que nous avons fait dans un autre document quelles variables il fallait utiliser pour minimiser les problèmes de multicolinéarité et quelles variables conserver. Pour débuter, on utilise donc les variables: "radius", "texture", "smoothness", "compactness", "concavity" et "symmetry" comme variables explicatives dans le modèle linéaire généralisé.

In [ ]:
selected_vars = tags[[1, 2, 5, 6, 7, 9]]
Term(:diagnosis) ~ sum(term.(selected_vars))
function trainGlmWith(df, formula=Term(:diagnosis) ~ sum(term.(selected_vars)))
    return glm(formula, df,  Bernoulli(), LogitLink())
end

trainGlmWith(data)

## Estimation par maximum de la vraisemblance du modèle de régression logistique
Pour voir la performance de ce modèle, on réalisera aussi une validation croisée.

In [ ]:
function glm_split_train_test(df, index)
    train = df[Not(index[1]:index[2]), :]
    test = df[index[1]:index[2], :]
    return train, test
end

function glm_kcross(df, k, formula=Term(:diagnosis) ~ sum(term.(selected_vars)))
    indexes = findAllIndexes(size(df, 1), k)
    θ̂  = Float64[]
    for interval = indexes
        train, test = glm_split_train_test(df, interval)
        M = trainGlmWith(train, formula)
        θ̂ᵢ = GLM.predict(M, test)
        append!(θ̂, θ̂ᵢ)
    end
    return θ̂
end

θ̂ = glm_kcross(data, 15)

## Mesure de la qualité du modèle

Supposons que l'on prédit que la tumeur est maligne si θ̂ > 1/2.

In [ ]:
# Le taux de bonnes prédictions
ŷ = zeros(Int64,length(data.diagnosis))
ŷ[θ̂ .> 1/2] .= 1 

println("Le taux de bonnes prédictions est de ", round(correctrate(data.diagnosis, ŷ), digits=3),".")

In [ ]:
# Calcul du taux de vrais positifs et de faux positifs pour un seuil de 1/2.

r = roc(data.diagnosis, θ̂, 1/2)

println("La sensibilité est de ", round(recall(r), digits=3))
println("La spécificité est de ", round(precision(r), digits=3))
println("Le score F1 est de ", round(f1score(r), digits=3))

In [ ]:
# Calcul de l'aire sous la courbe ROC
A = auc(data.diagnosis, θ̂)

In [ ]:
# Affichage de la courbe ROC
rocplot(data.diagnosis, θ̂)

## Recherche du meilleur modèle

Puisqu'on n'a que 10 variables,  on peut se permettre de tester toutes les possiblités. Cela devrait prendre entre 5 et 10 minutes! On se servira de l'aire sous la courbe ROC pour définir le meilleur modèle.

In [ ]:
models = DataFrame(Variables = Array{Symbol}[], AUC = Float64[])

# Premier modèle : aucune variable
push!(models, [[], 0.5])

# On teste tous les modèles possibles
for var in tags
    for model in eachrow(models)
        vars = copy(model[:Variables])
        push!(vars, var)

        formula = Term(:diagnosis) ~ sum(term.(vars))
        θ̂ = glm_kcross(data, 15, formula)
        AUC = auc(data.diagnosis, θ̂)

        push!(models, [vars, AUC])
    end
end

# Le meilleur modèle est sélectionné avec la plus grande aire sous la courbe ROC
sort!(models, :AUC, rev=true)
top10 = first(models, 10) 

On se servira donc du modèle incluant toutes les variables sauf radius, compactness et concavity.

In [ ]:
best_model = models[1, :]

In [ ]:
θ̂ = glm_kcross(data, 10, Term(:diagnosis) ~ sum(term.(best_model.Variables)))

In [ ]:
rocplot(data.diagnosis, θ̂)

## Choix du seuil

In [ ]:
plot(x -> f1score(roc(data.diagnosis, θ̂, x)), 0, 1)

In [ ]:
plot(x -> correctrate(θ̂ .> x, data.diagnosis), 0, 1)

Tant qu'il reste autour de 0.5, le seuil ne smeble pas avoir une influence majeure sur la qualité de nos prédictions, du moins sur notre ensemble de validation. Nous allon donc garder un seuil non biaisé de 0.5

In [ ]:
println("Le taux de bonnes prédictions est de ", round(correctrate(θ̂ .> 1/2, data.diagnosis), digits=3),".")

# Section 5: Comparaison de modèles

Dans cette partie, nous nous sommes demandé si l'on pouvait utiliser plusieurs modèles pour améliorer nos prédictions, en comparant les résultats qui diffèrent entre eux.

En comparant deux modèles, il y a 3 cas de figure :
- Les modèles ont tous les deux raison
- Les modèles ont tous les deux tort
- Les modèles se contredisent

En analysant les données corresondant à chacun de ces cas, on pourrait savoir quand prioriser un modèle par rapport à l'autre.

Pour pouvoir quantifier l'incertitude du modèle KNN, on lui fera plutôt donner la moyenne du diagnostic de ses plus proches voisin que la prédiction finale.

In [ ]:
function predict_counts(k, X_train, y_train, X_test) #works best with odd k    
    nb_data = size(X_train)[1]
    
    distances = []
    for elem in 1:nb_data
        push!(distances, calculateDist(X_test, X_train[elem, :]))
    end
    
    indexes_distances = findIndexesNSmallest(k, distances)
    
    nb_0 = 0
    for index in indexes_distances
        if (y_train[index] == 0)
            nb_0 += 1
        end
    end
    
    return (nb_0, k-nb_0)
    
end

function knn_counts(k, X_train, y_train, X_test)
    ans = []
    for elem in 1:size(X_test)[1]
        push!(ans, predict_counts(k, X_train, y_train, X_test[elem, :]))
    end
    return ans
end

## Génération des prédictions avec validation croisée

In [ ]:
KNN_predictions = []
bayesian_predictions = DataFrame(id = Int64[], predictions = Float64[])

nb_blocks = 15
indexes = findAllIndexes(size(X_train)[1], nb_blocks)


for i=1:nb_blocks
    m_train, b_train, valid = bayesian_split_data(i, nb_blocks)
    
    # Prédictions avec toutes les variables
    results = bayesian_predict(m_train, b_train, valid, 1:10)
    append!(bayesian_predictions, results)
    
    sub_X_train, sub_X_test, sub_y_train, true_y_test = split_train_test(X_train, y_train, indexes[i])
    sub_y_test = knn_counts(5, sub_X_train, sub_y_train, sub_X_test)
    append!(KNN_predictions, sub_y_test)

end

In [ ]:
# Ces prédictions sont dans l'ordre des indexes
# Les tuples sont donnés sous la forme [voisin_benin, voisin_malin]
KNN_predictions

In [ ]:
# La prédiction est la moyenne des diagnostics
KNN_predictions = map(tuple -> tuple[2] / (tuple[1] + tuple[2]), KNN_predictions)

In [ ]:
sort!(bayesian_predictions, :id)
first(bayesian_predictions, 5)

In [ ]:
#GLM
glm_predictions = glm_kcross(data, 15, Term(:diagnosis) ~ sum(term.(best_model.Variables)))

On peut maintenant rassembler ces prédictions dans nos données

In [ ]:
data = CSV.read("train.csv")
insertcols!(data, 13, :knn => KNN_predictions)
insertcols!(data, 14, :bayesian => bayesian_predictions.predictions)
insertcols!(data, 15, :glm => glm_predictions)
names(data)

In [ ]:
correctrate(data.knn .> 0.5, data.bayesian .> 0)

In [ ]:
correctrate(data.knn .> 0.5, data.glm .> 0.5)

In [ ]:
correctrate(data.glm .> 0.5, data.bayesian .> 0)

Les modèles sont en désaccord sur certaines valeurs. C'est une bonne nouvelle, car on peut trouver une manière de les départager, et ainsi d'obtenir une meilleure précision sur nos données.

On commencera par voir sur combien de données nos modèles sont en désaccord, et sur combien ils se trompent tous.

In [ ]:
data_disagree = filter(row -> !((row.knn > 0.5) == (row.bayesian > 0.5) == (row.glm > 0.5)), data)
select(data_disagree, [:knn, :bayesian, :glm, :diagnosis])

In [ ]:
data_all_wrong = filter(row -> (row.knn > 0.5) == (row.bayesian > 0.5) == (row.glm > 0.5)
                                && (row.diagnosis == 1) != (row.knn > 0.5) , data)
select(data_all_wrong, [:knn, :bayesian, :glm, :diagnosis])

Dans les données sujettes à désaccord, on peut remarquer que les modèles sont généralement assez peu sûrs d'eux (la probabilité, ou le log de la probabilité, sont proches du seuil de séparation). Ce n'est pas toujours le cas quand ils se trompent tous, où certaines valeurs extrêmes échappent complètement à leur contrôle.

Une première approche pourrait être de faire voter ces différents modèles, afin de faire varier les sources d'erreurs induites par nos différentes approches.

In [ ]:
function vote(df::DataFrame, knn_th, bayes_log_th, glm_th)
    vote_results = Int64[]
    for row in eachrow(df)
        knn_pred = row.knn > knn_th
        bayes_pred = row.bayesian > bayes_log_th
        glm_pred = row.glm > glm_th
        vote = mean([knn_pred, bayes_pred, glm_pred]) .> 0.5
        push!(vote_results, vote)
    end
    return vote_results
end

correctrate(vote(data, 0.5, 0.5, 0.5), data.diagnosis)

On a améliorer un peu notre qualité de prédiction, mais pas de beaucoup! Une autre option pourrait être d'attribuer un certain coefficient à chaque modèle, ce qu'on peut faire en utilisant la régression. Cette approche n'est pas rigoureuse, puisqu'on utilise deux fois les données, mais nous sommes désespérés d'obtenir un bon score sur kaggle :-(

In [ ]:
M = glm(@formula(diagnosis ~ knn + bayesian + glm), data,  Bernoulli(), LogitLink())

In [ ]:
correctrate(GLM.predict(M, data) .> 0.5, data.diagnosis)

C'est un peu mieux! On peut entrainer notre modèle sur l'ensemble des données et prédire l'ensemble de test à partir de ce nouveau modèle.

In [ ]:
data = CSV.read("train.csv")
test = CSV.read("test.csv")
insertcols!(test, 12, :knn => map(tuple -> tuple[2] / (tuple[1] + tuple[2]), knn_predict(data, test)))
insertcols!(test, 13, :bayesian => sort!(bayesian_predict(malign, benign, test, 1:10), :id).predictions)
glm_model = glm(Term(:diagnosis) ~ sum(term.(best_model.Variables)), data, Bernoulli(), LogitLink())
insertcols!(test, 14, :glm => GLM.predict(glm_model, test))
names(test)

In [ ]:
final_predictions = GLM.predict(M, test)
CSV.write("blend.csv", DataFrame(id = test.id, diagnosis = final_predictions .> 0.5))

88 % :-/